# LangChain - Multiple retrievers

## Install dependencies

In [1]:
!pip install langchain
!pip install openai
!pip install chromadb
!pip install tiktoken
!pip install python-dotenv

## Load environment variables

Change the path if necessary (read `README.md` first and follow the instructions to setup the `.env` file):

In [2]:
import os
os.chdir("/home/jovyan/work/")

In [3]:
%load_ext dotenv
%dotenv

Disable warnings:

In [4]:
import warnings
warnings.filterwarnings('ignore')

## Load a repository

In [5]:
from langchain.document_loaders.text import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
import urllib.request

In [6]:
embeddings = OpenAIEmbeddings(disallowed_special=())

text_splitter = CharacterTextSplitter(
    separator = "\n\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
)

In [7]:
db = Chroma(embedding_function=embeddings)

In [8]:
from langchain.vectorstores.base import VectorStoreRetriever


def ingest_book(book_url: str, book_name: str, db: Chroma) -> VectorStoreRetriever:
    file_path = f"/tmp/{book_name}.txt"
    urllib.request.urlretrieve(book_url, file_path)

    loader = TextLoader(file_path)
    documents = loader.load()

    texts = text_splitter.split_documents(documents)

    chunks = [doc.page_content for doc in texts]
    metadatas = [doc.metadata for doc in texts]
    db.add_texts(texts=chunks, metadatas=metadatas)

    return db.as_retriever(
        search_type="similarity",
        search_kwargs={
            "k": 6,
            "filter": {
                "source": file_path,
            }
        },
    )

In [9]:
retriever1 = ingest_book("https://www.gutenberg.org/files/55317/55317-0.txt", "Meditations", db)

Created a chunk of size 1079, which is longer than the specified 1000
Created a chunk of size 1067, which is longer than the specified 1000
Created a chunk of size 4361, which is longer than the specified 1000
Created a chunk of size 3659, which is longer than the specified 1000
Created a chunk of size 1232, which is longer than the specified 1000
Created a chunk of size 1016, which is longer than the specified 1000
Created a chunk of size 1066, which is longer than the specified 1000
Created a chunk of size 1317, which is longer than the specified 1000
Created a chunk of size 2225, which is longer than the specified 1000
Created a chunk of size 1557, which is longer than the specified 1000
Created a chunk of size 1488, which is longer than the specified 1000
Created a chunk of size 1235, which is longer than the specified 1000
Created a chunk of size 2929, which is longer than the specified 1000
Created a chunk of size 1019, which is longer than the specified 1000
Created a chunk of s

In [10]:
retriever2 = ingest_book("https://www.gutenberg.org/ebooks/12096.txt.utf-8", "Bushido", db)

Created a chunk of size 1069, which is longer than the specified 1000
Created a chunk of size 1812, which is longer than the specified 1000
Created a chunk of size 1581, which is longer than the specified 1000
Created a chunk of size 1693, which is longer than the specified 1000
Created a chunk of size 2451, which is longer than the specified 1000
Created a chunk of size 1075, which is longer than the specified 1000
Created a chunk of size 1069, which is longer than the specified 1000
Created a chunk of size 1086, which is longer than the specified 1000
Created a chunk of size 1846, which is longer than the specified 1000
Created a chunk of size 1960, which is longer than the specified 1000
Created a chunk of size 1464, which is longer than the specified 1000
Created a chunk of size 1049, which is longer than the specified 1000
Created a chunk of size 2143, which is longer than the specified 1000
Created a chunk of size 1411, which is longer than the specified 1000
Created a chunk of s

## Router

In [11]:
from langchain.chains.router import MultiRetrievalQAChain
from langchain.chat_models import ChatOpenAI

**Note:** change between models, or tweak splitter and retrievers parameters, if you have issues. It works well with GPT-4.

In [12]:
llm = ChatOpenAI(temperature=0)
#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
#llm = ChatOpenAI(model="gpt-4", temperature=0)

In [13]:
retriever_infos = [
    {
        "name": "meditations",
        "description": "Good for answering questions about Meditations by Marcus Aurelius",
        "retriever": retriever1,
    },
    {
        "name": "bushido",
        "description": "Good for answering questions about Bushido book",
        "retriever": retriever2,
    },
]

In [14]:
qa = MultiRetrievalQAChain.from_retrievers(llm, retriever_infos)

## Test router

In [15]:
qa("How the Bushido approachs the concept of personal honor?")

{'input': 'How the Bushido approachs the concept of personal honor?',
 'query': 'How does the Bushido approach the concept of personal honor?',
 'result': "Bushido places a great emphasis on personal honor. It teaches to bear and face all calamities and adversities with patience and a pure conscience. It believes that true honor lies in fulfilling Heaven’s decree and no death incurred in so doing is ignominious. It also teaches that it is cowardly to avoid what Heaven has in store. The misuse of one's weapon for undeserved occasions is seen as dishonorable. A self-possessed man knows the right time to use it, and such times come but rarely. The ultimate ideal of knighthood, according to Bushido, is peace. However, the fear of disgrace was so great that it often assumed a morbid character and led to unnecessary strife and loss of innocent lives."}

In [16]:
qa("How Marcus Aurelio approachs the concept of personal honor?")

{'input': 'How Marcus Aurelio approachs the concept of personal honor?',
 'query': 'How does Marcus Aurelius approach the concept of personal honor in Meditations?',
 'result': "Marcus Aurelius approaches the concept of personal honor in Meditations by emphasizing the importance of virtues such as modesty, manliness, piety, and generosity. He suggests that personal honor is not found in external achievements or recognition, but in living according to one's principles and nature. He encourages the reader to live each moment as if it were their last, with dignity, kindness, freedom, and justice. He also warns against seeking happiness in the approval of others, instead urging the reader to honor themselves. He suggests that personal honor is tied to acting in a way that is just, temperate, courageous, and free."}

## Agent

In [17]:
from langchain.agents import initialize_agent, Tool

In [18]:
tools = [
    Tool(
        name = "Meditations",
        func=retriever1.get_relevant_documents,
        description="Useful for answering questions about Meditations by Marcus Aurelius."
    ),
    Tool(
        name = "Bushido",
        func=retriever1.get_relevant_documents,
        description="Useful for answering questions about Bushido book."
    ),
]

In [19]:
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

## Test agent

In [20]:
print(agent.run("What is the perspective of Marcus Aurelius and Bushido on the nature of suffering and adversity, and how do they recommend dealing with these challenges?"))



> Entering new  chain...
I need to first understand Marcus Aurelius's perspective on suffering and adversity from Meditations, and then understand the perspective of Bushido on the same topic.
Action: Meditations
Action Input: What is Marcus Aurelius's perspective on suffering and adversity, and how does he recommend dealing with these challenges?
Observation: [Document(page_content='36. Do not confound yourself by considering the whole of life, and by\ndwelling upon the multitude and greatness of the pains and troubles to\nwhich you may probably be exposed. As each presents itself ask\nyourself: Is there anything intolerable and insufferable in this? You\nwill be ashamed to own it. And then recollect that it is neither the\npast nor the future that can oppress you, but always the present\nonly. And the ills of the present will be much diminished if you\nrestrict it within its own proper bounds, and take your soul to task\nif it cannot bear up even against this one thing.', metadata=

In [21]:
print(agent.run("How do Marcus Aurelius and the principles of Bushido approach the pursuit of inner peace and tranquility in the midst of external chaos and conflict?"))



> Entering new  chain...
I need to understand the perspectives of both Marcus Aurelius in Meditations and the principles of Bushido on the topic of inner peace and tranquility in the midst of chaos and conflict.
Action: Meditations
Action Input: What does Marcus Aurelius say about achieving inner peace and tranquility in the midst of chaos and conflict?
Observation: [Document(page_content='24. "Do few things," says the philosopher, "if you would have quiet."\nThis is perhaps a better saying, "Do what is necessary, do what the\nreason of the being that is social in its nature directs, and do it in\nthe spirit of that direction." By this you will attain the calm that\ncomes from virtuous action, and that calm also which comes from having\nfew things to do. Most things you say and do are not necessary. Have\ndone with them, and you will be more at leisure and less perturbed. On\nevery occasion, then, ask yourself the question, Is this thing not\nunnecessary? And put away not only unnece

In [22]:
print(agent.run("In what ways do Marcus Aurelius and the principles of Bushido emphasize the importance of constant self-improvement and lifelong learning?"))



> Entering new  chain...
Both Meditations and Bushido may have insights on this topic. I should first check Meditations for Marcus Aurelius's views on self-improvement and lifelong learning.
Action: Meditations
Action Input: self-improvement and lifelong learning
Observation: [Document(page_content='11. Acquire a method of perceiving how all things change into one\nanother. Pursue this branch of Philosophy and continually exercise\nyourself therein. There is nothing so proper as this for cultivating\ngreatness of mind. He who does so has already put off the body; and,\nhaving realized how soon he must depart from among men and leave all\nearthly things behind him, he resigns himself entirely to justice in\nall his own actions, and to the law of the Universe in everything else\nwhich happens. As for what any one may say or think of him or do\nagainst him, he gives it not a thought, but contents himself with\nthese two things: to do justly what he has in hand, and to love the\nlot appo

In [23]:
print(agent.run("What are the differences between how Marcus Aurelius and the Bushido code address the role of emotions in decision-making and daily life?"))



> Entering new  chain...
I need to compare the perspectives of Marcus Aurelius in Meditations and the Bushido code on the role of emotions in decision-making and daily life.
Action: Meditations
Action Input: What does Marcus Aurelius say about the role of emotions in decision-making and daily life?
Observation: [Document(page_content='36. No man can rob you of your liberty of action; as has been said by\nEpictetus.\n\n37. He tells us also that we must find out the true art of assenting;\nand in treating of our impulses he says that we must be vigilant in\nrestraining them, that they may act with proper reservation, with\npublic spirit, with due sense of proportion; also that we should\nrefrain utterly from sensual passion; and not be restive in matters\nwhere we have no control.\n\n38. The contention is not about any chance matter, said he, but as to\nwhether we are insane or sane.\n\n39. What do you desire? says Socrates. To have the souls of rational\nbeings or of irrational? Ratio